In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, MaxPool2D, Dropout, Flatten, Conv2D, GlobalAveragePooling2D, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from random import shuffle, choice
from scipy import stats as st

from collections import deque
import glob

In [2]:
def gather_data(num_samples):
    
    global rock, paper, scissors, nothing
#     global class
    cap = cv2.VideoCapture(0)
    
    trigger = False
    counter = 0
    
    box_size = 234
    
    width = int(cap.get(3))
    
    
    while True:
        ret , frame = cap.read()
        frame = cv2.flip(frame, 1)
        
        if counter == num_samples:
            trigger = not trigger
            counter = 0
            
        cv2.rectangle(frame, (width - box_size, 0), (width, box_size), (0, 250, 150), 2)
        
        cv2.namedWindow("Collecting images", cv2.WINDOW_NORMAL)
        
        if trigger:
            roi = frame[5: box_size-5 , width-box_size + 5: width -5]
            eval(class_name).append([roi, class_name])
            counter += 1
            text = "Collected Samples of {}: {}".format(class_name, counter)
        else:
            text = "Press 'r' to collect rock samples, 'p' for paper, 's' for scissor and 'n' for nothing"
            
            
        cv2.putText(frame, text, (3, 350), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 1, cv2.LINE_AA)
        cv2.imshow("Collecting images", frame)
        k = cv2.waitKey(1)
        if k == ord('r'):
            trigger = not trigger
            class_name = 'rock'
            rock = []
        
        if k == ord('p'):
            trigger = not trigger
            class_name = 'paper'
            paper = []
            
        if k == ord('s'):
            trigger = not trigger
            class_name = 'scissors'
            scissors = []
            
        if k == ord('n'):
            trigger = not trigger
            class_name = 'nothing'
            nothing = []
            
        if k == ord('q'):
            break
            
    cap.release()
    cv2.destroyAllWindows()


In [3]:
no_of_samples = 200
gather_data(no_of_samples)


In [4]:
data = [rock, paper, scissors, nothing]
labels = []
images = []


for i in data:
    for j in i:
        labels.append(j[1])
        images.append(j[0])
        
images = np.array(images, dtype="float") / 255.0
encoder = LabelEncoder()
labels_coded = encoder.fit_transform(labels)
hot_labels = to_categorical(labels_coded, 4)

trainX, testX, trainY, testY = train_test_split(images, hot_labels, test_size=0.1, random_state=50)

images = []
data = []

In [5]:
N_mobile = tf.keras.applications.NASNetMobile( input_shape = (224,224,3), include_top = False, weights = "imagenet")

N_mobile.trainable = False

x = N_mobile.output

x = GlobalAveragePooling2D()(x)

x = Dense(712, activation = "relu")(x)
x = Dropout(0.40)(x)
preds = Dense(4,activation='softmax')(x)
model = Model(inputs=N_mobile.input, outputs=preds)

In [6]:
augment = ImageDataGenerator(
    rotation_range = 30,
    zoom_range = 0.25,
    width_shift_range = 0.10,
    height_shift_range = 0.10,
    shear_range = 0.10,
    horizontal_flip = True,
    fill_mode = "nearest")

In [7]:
model.compile(optimizer = Adam(learning_rate = 0.0001),loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
history = model.fit(x = augment.flow(trainX, trainY, batch_size = 20), validation_data = (testX, testY),
                    steps_per_epoch= len(trainX) // 20, epochs=15)


Epoch 1/15
36/36 [==============================] - 73s 1s/step - loss: 0.8382 - accuracy: 0.7250 - val_loss: 0.3128 - val_accuracy: 0.9625
Epoch 2/15
36/36 [==============================] - 50s 1s/step - loss: 0.3250 - accuracy: 0.9389 - val_loss: 0.1381 - val_accuracy: 0.9750
Epoch 3/15
36/36 [==============================] - 65s 2s/step - loss: 0.1913 - accuracy: 0.9681 - val_loss: 0.0885 - val_accuracy: 0.9750
Epoch 4/15
36/36 [==============================] - 52s 1s/step - loss: 0.1321 - accuracy: 0.9694 - val_loss: 0.0748 - val_accuracy: 0.9750
Epoch 5/15
36/36 [==============================] - 52s 1s/step - loss: 0.1146 - accuracy: 0.9750 - val_loss: 0.0547 - val_accuracy: 0.9875
Epoch 6/15
36/36 [==============================] - 55s 2s/step - loss: 0.0792 - accuracy: 0.9889 - val_loss: 0.0315 - val_accuracy: 1.0000
Epoch 7/15
36/36 [==============================] - 60s 2s/step - loss: 0.0695 - accuracy: 0.9875 - val_loss: 0.0353 - val_accuracy: 0.9875
Epoch 8/15
36/36 [==

In [2]:
# model.save("Gesture.h5")
model = load_model("Gesture.h5")

In [3]:
label_names = ["Nothing", "Paper", "Rock", "Scissors"]
cap = cv2.VideoCapture(0)
box_size = 234
width = int(cap.get(3))


while True:
    ret, frame = cap.read()
    width = int(cap.get(3))
    
    frame = cv2.flip(frame, 1)
    
    cv2.rectangle(frame, (width - box_size, 0), (width, box_size), (0, 250, 150), 2)
    cv2.namedWindow("Rock Paper Scissors", cv2.WINDOW_NORMAL)
    roi = frame[5: box_size-5 , width-box_size + 5: width -5]
    roi = np.array([roi]).astype('float64') / 255.0

    
#     roi = cv2.resize(roi, (224,224))
#     roi = roi.reshape(-1,224,224,3)
    
    pred = model.predict([roi])
    target_index = np.argmax(pred[0])
    prob = np.max(pred[0])
    
    cv2.putText(frame, "prediction: {} {:.2f}%".format(label_names[np.argmax(pred[0])], prob*100 ),
                (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.90, (0, 0, 255), 2, cv2.LINE_AA)


    
    
    cv2.imshow("frame", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()
    

1/1 [==============================] - 0s 91ms/step


In [4]:
def findout_winner(user_move, Computer_move):
    if user_move == Computer_move:
        return "Tie"   
    
    elif user_move == "Rock" and Computer_move == "Scissors":
        return "User"

    elif user_move == "Rock" and Computer_move == "Paper":
        return "Computer"

    elif user_move == "Scissors" and Computer_move == "Rock":
        return "Computer"  

    elif user_move == "Scissors" and Computer_move == "Paper":
        return "User"
    
    elif user_move == "Paper" and Computer_move == "Rock":
        return "User"
    
    elif user_move == "Paper" and Computer_move == "Scissors":
        return "Computer"


In [13]:
def show_winner(user_score, computer_score):
    if user_score > computer_score:
        img = cv2.imread("Path/You Win.jpg")
        img = cv2.resize(img, (224,224))
        
    elif computer_score > user_score:
        img = cv2.imread("Path/You Loose.jpg")
        img = cv2.resize(img, (224,224))
    else:
        img = cv2.imread("Path/Draw.jpg")
        img = cv2.resize(img, (224,224))
    cv2.putText(img, "Press 'ENTER' to play again, else exit",
                (120, 180), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 3, cv2.LINE_AA)
        
#     cap.release()
    cv2.destroyAllWindows()
        
    cv2.imshow("RockPaperScissor", img)
    
    k = cv2.waitKey(0)
    
    if k == 13:
        cv2.destroyAllWindows()
        return True
    else:
        return False

In [14]:
def display_computer_move(computer_move_name, frame):
    icon = cv2.imread("Path/{}.jpg".format(computer_move_name))
    icon = cv2.resize(icon, (224,224))
    
    roi = frame[0:224, 0:224]
    mask = icon[:, :, -1]
    
    mask = cv2.threshold(mask, 1, 255, cv2.THRESH_BINARY)[1]
    icon_bgr = icon[:,:,:3]
    img1_bg = cv2.bitwise_and(roi, roi, mask = cv2.bitwise_not(mask))
    img2_fg = cv2.bitwise_and(icon_bgr, icon_bgr, mask = mask)
    
    combined = cv2.add(img1_bg, img2_fg)
    frame[0:224, 0:224] = combined
    return frame


In [16]:
cap = cv2.VideoCapture(0)
box_size = 234
width = int(cap.get(3))

attempts = 5

computer_move_name = "nothing"
final_user_move = "nothing"
label_names = ['nothing', 'Paper', 'Rock', 'Scissors']
computer_score , user_score = 0, 0
rect_color = [0,0,255]
hand_inside = False
total_attempts = attempts
confidence_threshold = 0.70
smooth_factor = 5

de = deque(["nothing"]*5, maxlen = smooth_factor)


while True:
    ret, frame = cap.read()
    
    if not ret:
        break
        
    frame = cv2.flip(frame, 1)
    
    cv2.namedWindow("Rock Paper Scissors", cv2.WINDOW_NORMAL)
    
    roi = frame[5: box_size-5 , width-box_size + 5: width -5]
    roi = np.array([roi]).astype('float64') / 255.0
    
    pred = model.predict(roi)
    move_code = np.argmax(pred[0])
    user_move = label_names[move_code]
    prob = np.max(pred[0])
    
    if prob >= confidence_threshold:
        de.appendleft(user_move)
        
        try:
            final_user_move = st.mode(de)[0][0]
            
        except StatisticsError:
            print('Stats error')
            continue

        if final_user_move != "nothing" and hand_inside == False:
            hand_inside = True
            computer_move_name = choice(['Rock', 'Paper', 'Scissors'])
            winner = findout_winner(final_user_move, computer_move_name)
            display_computer_move(computer_move_name, frame)
            total_attempts -= 1

            if winner == "Computer":
                computer_score +=1
#                 print(computer_score)
                rect_color = (0, 0, 255)
                
            elif winner == "User":
                user_score += 1;
                rect_color = (0, 250, 0) 
                
            elif winner == "Tie":
                rect_color = (255, 250, 255)
                
            if total_attempts == 0:
                play_again = show_winner(user_score, computer_score)
                
                if play_again:
                    user_score, computer_score, total_attempts = 0, 0, attempts
                    
                else:
                    break
                    
        elif final_user_move != "nothing" and hand_inside == True:
            display_computer_move(computer_move_name, frame)
            
        elif final_user_move == 'nothing':           
            hand_inside = False
            rect_color = (255, 0, 0)
            
    cv2.putText(frame, "Your Move: " + final_user_move,
                (420, 270), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1, cv2.LINE_AA)
    
    cv2.putText(frame, "Computer's Move: " + computer_move_name,
                (2, 270), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1, cv2.LINE_AA)

    cv2.putText(frame, "Your Score: " + str(user_score),
                (420, 300), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1, cv2.LINE_AA)
    
    cv2.putText(frame, "Computer Score: " + str(computer_score),
                (2, 300), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1, cv2.LINE_AA)
    
    cv2.putText(frame, "Attempts left: {}".format(total_attempts), (190, 400), cv2.FONT_HERSHEY_COMPLEX, 0.7,
                (100, 2, 255), 1, cv2.LINE_AA)   

    cv2.rectangle(frame, (width - box_size, 0), (width, box_size), rect_color, 2)
    
    cv2.imshow("Rock Paper Scissors", frame)

    k = cv2.waitKey(10)
    if k == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 99ms/step


1/1 [==============================] - 0s 86ms/step
